In [1]:
import selenium.webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import atexit
import os
import pandas as pd
from glob import glob

In [2]:
#전역 변수 선언
TIMEOUT = 10
ROOT_DIR = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
DOWNLOAD_DIR = f'{ROOT_DIR}\\download\\riss'
print(DOWNLOAD_DIR)

#사이트 주소
URL = 'http://www.etlandmall.co.kr/pc/product/product.do?prdMstCd=S2456232'
#URL = 'http://www.etlandmall.co.kr/pc/product/product.do?prdMstCd=S2542730'

#로그인 정보 (본인 정보를 입력해야 함)
USER = 'millyseon77'
PASSWD = 'goghks-0906'

c:\development\workspaces\python\weekend\backup_20210105\download\riss


In [3]:
#chrome webdriver를 생성한다.
# - download_path: 파일을 다운로드 받을 위치
# - headless: True이면 브라우저 화면 없이 background로 처리
def create_chrome(download_path, headless=False) :
    options = ChromeOptions()
    if headless:
        options.add_argument('headless')
        options.add_argument('--disable-gpu')
    options.add_experimental_option('prefs', {
        'download.default_directory' : download_path,
        'download.prompt_for_download' : False
    })

    chrome = Chrome(chrome_options = options)

    if headless :
        _enable_download_on_chrome_headless(chrome, download_path)

    #프로세스 종료시 크롬을 종료시키는 로직
    atexit.register(_close_chrome(chrome))
    
    return chrome

#headless=True 일때 다운로드 안되는(크롬 보안문제) 문제 해결
def _enable_download_on_chrome_headless(browser, download_dir) :
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')

    params = {
        'cmd': 'Page.setDownloadBehavior',
        'params': {
            'behavior': 'allow',
            'downloadPath': download_dir
        }
    }
    browser.execute("send_command", params)

#크롬 종료~
def _close_chrome(chrome) :
    def close() :
        chrome.close()
    return close

def chrome_wait(wait, condition):
    try:
        element = wait.until(condition)
        print("[DEBUG] 로딩 완료")
        return element
    except TimeoutException:
        print(f"[DEBUG] 페이지 로딩 시간 초과")

###여기서 부터 시작

In [4]:
browser = create_chrome(download_path=DOWNLOAD_DIR)
wait = WebDriverWait(browser, TIMEOUT)
browser.get(URL)

chrome_wait(wait, EC.presence_of_element_located((By.CLASS_NAME, "menu_list")))

browser.execute_script("javascript:goLoginPage();")

print("[INFO] 로그인 버튼 클릭")
chrome_wait(wait, EC.element_to_be_clickable((By.CLASS_NAME, "btn_login")))

e_id = browser.find_element_by_id("MEM_MST_MEM_ID")
e_id.send_keys(USER)

e_passwd = browser.find_element_by_id("MEM_MST_WEB_PWD")
e_passwd.send_keys(PASSWD)
e_passwd.send_keys(Keys.RETURN)

[DEBUG] 로딩 완료
[INFO] 로그인 버튼 클릭
[DEBUG] 로딩 완료


In [ ]:
#URL = 'http://www.etlandmall.co.kr/pc/product/product.do?prdMstCd=S2456232'
URL = 'http://www.etlandmall.co.kr/pc/product/product.do?prdMstCd=S2542730'
browser.get(URL)

In [5]:
def get_e_buy() :
    try : 
        return browser.find_element_by_xpath('//*[@id="cartForm"]//a[text() = "구매하기"]')
    except NoSuchElementException :
        return None


In [68]:
print(get_e_buy())

<selenium.webdriver.remote.webelement.WebElement (session="80988297ca9626aad9e72ae1c7c968aa", element="07c8a7d8-458b-4e86-871a-9c0e73d2a378")>


In [87]:
##한번에
chrome_wait(wait, EC.element_to_be_clickable((By.XPATH, '//*[@id="cartForm"]//a[text() = "구매하기"]')))
e_buy = browser.find_element_by_xpath('//*[@id="cartForm"]//a[text() = "구매하기"]')
e_buy.click()

chrome_wait(wait, EC.element_to_be_clickable((By.ID, 'pasteOrderInfo')))

e_addr = browser.find_element_by_id("pasteOrderInfo")
e_addr.click()
e_payment = browser.find_element_by_id("paymentBtn_451")
e_payment.click()

e_card = browser.find_element_by_xpath('//div[@data-id="EP_card_cd1"]/button[text()="선택"]')
e_card.click()

e_card_2 = browser.find_element_by_xpath('//li/a[@code="031" and text() = "삼성카드"]')
e_card_2.click()

time.sleep(0.5)

browser.find_element_by_id('ORDER_CONFIRM').click()

chrome_wait(wait, EC.element_to_be_clickable((By.ID, 'goPayBtn')))
browser.execute_script("javascript:paymentPrepare();")

[DEBUG] 로딩 완료


UnexpectedAlertPresentException: Alert Text: 상품명 [[예약판매/1/27일 순차발송]PlayStation 5 플레이스테이션5 본체 PS5]
판매 불가 상품이 포함되어 있습니다.
Message: unexpected alert open: {Alert text : 상품명 [[예약판매/1/27일 순차발송]PlayStation 5 플레이스테이션5 본체 PS5]
판매 불가 상품이 포함되어 있습니다.}
  (Session info: chrome=87.0.4280.88)


In [91]:
#Step 1

chrome_wait(wait, EC.element_to_be_clickable((By.XPATH, '//*[@id="cartForm"]//a[text() = "구매하기"]')))
e_buy = browser.find_element_by_xpath('//*[   @id="cartForm"]//a[text() = "구매하기"]')
e_buy.click()

[DEBUG] 로딩 완료


In [ ]:
#Step 2

chrome_wait(wait, EC.element_to_be_clickable((By.ID, 'pasteOrderInfo')))

e_addr = browser.find_element_by_id("pasteOrderInfo")
e_addr.click()
e_payment = browser.find_element_by_id("paymentBtn_451")
e_payment.click()

e_card = browser.find_element_by_xpath('//div[@data-id="EP_card_cd1"]/button[text()="선택"]')
e_card.click()

e_card_2 = browser.find_element_by_xpath('//li/a[@code="031" and text() = "삼성카드"]')
e_card_2.click()

In [ ]:
#Step 3

chrome_wait(wait, EC.element_to_be_clickable((By.ID, 'ORDER_CONFIRM')))

browser.find_element_by_id('ORDER_CONFIRM').click()
browser.execute_script("javascript:paymentPrepare();")